In [1]:
from collections import Counter
import dspy
from dspy import OpenAI, settings
from dspy.teleprompt import MIPROv2, BootstrapFewShot
from dspy.evaluate.evaluate import Evaluate
from dspy import ColBERTv2
from dsp.utils import print_message, normalize_text
from dotenv import load_dotenv
import os
load_dotenv("/media/uberdev/ddrv/gitFolders/python_de_learners_data/.env")

/media/uberdev/ddrv/telemetenv/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
/media/uberdev/ddrv/telemetenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [40]:
def f1_score_01(prediction, ground_truth):
    prediction_tokens = [normalize_text(elem) for elem in prediction.split("|")]
    ground_truth_tokens = [normalize_text(elem) for elem in ground_truth.split("|")]
    
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    
    if len(prediction_tokens) == len(ground_truth_tokens) == 0:
        print_message("\n#> F1 Metric: Rare edge case of len(prediction_tokens) == len(ground_truth_tokens) == 0.\n")
    
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    print(f"precision: {precision}, recall: {recall}, f1: {f1}") 
    return f1

Key Idea is to ping the LLM multiple times for the same prediction
and then get the f1_score of the prediction.

Keep track of the calls to LLMs inside phoenix using the instrumentation

    - Need to create a seperate project inside the Phoenix server for this exploration

    - Log the traces to that Project

    - Use the logs to further understand Metric functions

    - Move farther into OpenTelemetry

In [3]:
from openinference.instrumentation.dspy import DSPyInstrumentor
# instruments the internal calls in DSPy library
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
# help to get the span of http requests to the APIs
from opentelemetry.sdk import trace as trace_sdk
#processes the data collected from the spans
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.semconv.resource import ResourceAttributes
import phoenix as px

INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/uberdev/.phoenix
INFO:phoenix.inferences.inferences:Dataset: phoenix_inferences_02bbca05-adbd-41a6-84d9-ab6f6dbbbb58 initialized


In [4]:
endpoint = "http://127.0.0.1:6006/v1/traces"
# resource = Resource(attributes={})

In [5]:
client = px.Client(endpoint=endpoint)

INFO:httpx:HTTP Request: GET http://127.0.0.1:6006/v1/traces/arize_phoenix_version "HTTP/1.1 200 OK"


In [6]:
# https://docs.arize.com/phoenix/tracing/how-to-tracing/trace-a-deployed-app
resource = Resource(attributes={
    ResourceAttributes.PROJECT_NAME: 'bswfs-f1-score'
})

In [7]:
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)

In [8]:
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)

DSPyInstrumentor().instrument(skip_dep_check=True) # here where DSPy is instrumented

In [9]:
# Need to check if the dspy is writing logs
import os
llm = OpenAI(model='gpt-4o-mini',
             api_key=os.environ['OPENAI_API_KEY'],
             max_tokens=2000)

In [10]:
colbertv2_wiki17_abstracts = ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
settings.configure(lm=llm, rm=colbertv2_wiki17_abstracts)

In [11]:
llm("Let's see if you are logging to bswfs-f1-score")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


['It seems like you\'re referring to a specific logging or scoring system, possibly related to a project or application. However, I don\'t have the capability to access external systems or databases, including any logging systems like "bswfs-f1-score." If you have questions about logging, scoring, or any related topics, feel free to ask!']

In [12]:
def F1(prediction, answers_list):
    # list of answers are entering for a single prediction
    assert isinstance(answers_list, list)
    return max(f1_score_01(prediction, ans) for ans in answers_list)

def answer_match(prediction, answers, frac=1.0):
    return F1(prediction, answers) >= frac

def answer_f1_match_01(example, pred, trace=[], frac=0.95):
    assert isinstance(example.answer, (str, list))
    print(f"Looking at the traces, {trace}")
    if isinstance(example.answer, str):
        return answer_match(pred.answer, [example.answer], frac=frac)
    else:
        return answer_match(pred.answer, example.answer, frac=frac)


In [13]:
class NewsCategorization(dspy.Signature):
    news_body = dspy.InputField(desc="The body of the news to be categorized")
    answer = dspy.OutputField(desc="Should be 'fake' or 'real'")

class CoTCombined(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(NewsCategorization)
        self.history = []  # This will store the history of operations

    def forward(self, news_body):
        pred_list = []
        for news in news_body.split("|"):
            pred_one = self.prog(news_body=news)
            pred_list.append(pred_one.answer)
            self.history.append(f"Processed news: {news}, Prediction: {pred_one.answer}")
        return dspy.Prediction(answer="|".join(pred_list))

    def inspect_history(self, n=None):
        """
        Return the last n entries of the history. If n is None, return the entire history.
        """
        if n is None:
            return self.history
        else:
            return self.history[-n:]

In [17]:
prgm_under_test = CoTCombined()

In [18]:
prgm_under_test.dump_state()

[('prog', Predict(StringSignature(news_body -> rationale, answer
    instructions='Given the fields `news_body`, produce the fields `answer`.'
    news_body = Field(annotation=str required=True json_schema_extra={'desc': 'The body of the news to be categorized', '__dspy_field_type': 'input', 'prefix': 'News Body:'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': "Should be 'fake' or 'real'", '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)))]


{'prog': {'lm': None,
  'traces': [],
  'train': [],
  'demos': [],
  'signature_instructions': 'Given the fields `news_body`, produce the fields `answer`.',
  'signature_prefix': 'Answer:',
  'extended_signature_instructions': 'Given the fields `news_body`, produce the fields `answer`.',
  'extended_signature_prefix': 'Answer:'}}

In [14]:
class CustomExample:
    def __init__(self, news_body, answer):
        self.news_body = news_body
        self.answer = answer
        self._dict = {"news_body": news_body,
                      "answer": answer}

    def with_inputs(self, input_key):
        return self

    def inputs(self):
        return {"news_body": self.news_body}

    def items(self):
        return self._dict.items()

    def copy(self):
        return CustomExample(self.news_body, self.answer)

    def get(self, key, default=None):
        return self._dict.get(key, default)

    def __iter__(self):
        return iter(self._dict)

    def __contains__(self, key):
        return key in self._dict

    def __getitem__(self, key):
        return self._dict[key]

    def keys(self):
        return self._dict.keys()

    def values(self):
        return self._dict.values()


In [28]:
# preping the dataset from the train_fake_real_news.tsv
custom_trainset = []
custom_devset = []

with open('train_fake_real_news.tsv', 'r') as tsv:
    lines = tsv.readlines()
    for line in lines[1:21]:
        news, truth = line.split("\t")
        if truth.strip() == '0':
            custom_trainset.append(CustomExample(news, 'fake'))
        else:
            custom_trainset.append(CustomExample(news, 'real'))
    for line in lines[22:43]:
        news, truth = line.split("\t")
        if truth.strip() == '0':
            custom_devset.append(CustomExample(news, 'fake'))
        else:
            custom_devset.append(CustomExample(news, 'real'))

In [36]:
for x in custom_trainset[0:10]:
    print(x.items())

dict_items([('news_body', ' Courts Decide Conspiracy Nut Alex Jones Is Too Crazy To Raise His Own Kids (DETAILS)'), ('answer', 'fake')])
dict_items([('news_body', "U.S. Senator Menendez's corruption trial to proceed: judge"), ('answer', 'real')])
dict_items([('news_body', 'Search ends for bodies in Mexico City after earthquake'), ('answer', 'real')])
dict_items([('news_body', 'BUH-BYE! GLENN BECK Places Final Nail In His Coffin…And His Former Fans Won’t Miss Him [VIDEO]'), ('answer', 'fake')])
dict_items([('news_body', 'BREAKING: Michigan Native KID ROCK Announces He’s Running For US Senate'), ('answer', 'fake')])
dict_items([('news_body', 'Medicaid cuts coming in Trump budget: Washington Post'), ('answer', 'real')])
dict_items([('news_body', 'GOP MAJORITY SENATE FINALLY GETS IT RIGHT: Votes To Gut Obamacare And Defund Planned Parenthood'), ('answer', 'fake')])
dict_items([('news_body', 'Tillerson seeks to reassure worried Europe over Trump'), ('answer', 'real')])
dict_items([('news_bo

In [37]:
print("Starting to build the optimizer")
model_to_generate_prompts = llm
model_that_solves_task = CoTCombined()
your_defined_metric = answer_f1_match_01
num_new_prompts_generated = 10
prompt_generation_temperature = 0.7

Starting to build the optimizer


In [42]:
from dspy.evaluate import Evaluate
test_with_evaluator = Evaluate(devset=custom_devset[:5], display_progress=True, num_threads=1)
test_with_evaluator(model_that_solves_task, metric=your_defined_metric)

  0%|          | 0/5 [00:00<?, ?it/s]Looking at the traces, []
precision: 1.0, recall: 1.0, f1: 1.0
Average Metric: 1 / 1  (100.0):   0%|          | 0/5 [00:00<?, ?it/s]Looking at the traces, []
precision: 1.0, recall: 1.0, f1: 1.0
Average Metric: 2 / 2  (100.0):  20%|██        | 1/5 [00:00<00:00, 11.35it/s]Looking at the traces, []
precision: 1.0, recall: 1.0, f1: 1.0
Average Metric: 3 / 3  (100.0):  60%|██████    | 3/5 [00:00<00:00, 18.41it/s]Looking at the traces, []
precision: 1.0, recall: 1.0, f1: 1.0
Average Metric: 4 / 4  (100.0):  60%|██████    | 3/5 [00:00<00:00, 18.41it/s]Looking at the traces, []
precision: 1.0, recall: 1.0, f1: 1.0
Average Metric: 5 / 5  (100.0): 100%|██████████| 5/5 [00:00<00:00, 18.64it/s]

INFO:dspy.evaluate.evaluate:2024-08-17T12:54:26.728863Z [info     ] Average Metric: 5 / 5 (100.0%) [dspy.evaluate.evaluate] filename=evaluate.py lineno=200


100.0

In [38]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

config = dict(max_bootstrapped_demos=2,
              max_labeled_demos=4,
              num_candidate_programs=2,
              num_threads=6)

teleprompter = BootstrapFewShotWithRandomSearch(metric=your_defined_metric, **config)

Going to sample between 1 and 2 traces per predictor.
Will attempt to bootstrap 2 candidate sets.


In [19]:
# teleprompter = BootstrapFewShot(metric=your_defined_metric)
# print("Teleprompter assembled")

Teleprompter assembled


In [39]:
optimized_cotcomb = teleprompter.compile(model_that_solves_task,
                                    trainset=custom_trainset,
                                    valset=custom_devset)

Average Metric: 1 / 1  (100.0):   5%|▍         | 1/21 [00:01<00:25,  1.25s/it]

Looking at the traces, []


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Average Metric: 3 / 3  (100.0):  10%|▉         | 2/21 [00:01<00:14,  1.34it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Looking at the traces, []
Looking at the traces, []


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Average Metric: 5 / 5  (100.0):  19%|█▉        | 4/21 [00:02<00:06,  2.45it/s]

...

Looking at the traces, []
Looking at the traces, []


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Average Metric: 19 / 19  (100.0):  90%|█████████ | 19/21 [00:07<00:00,  2.79it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Average Metric: 20 / 20  (100.0):  90%|█████████ | 19/21 [00:07<00:00,  2.79it/s]

Looking at the traces, []
Looking at the traces, []


Average Metric: 21 / 21  (100.0): 100%|██████████| 21/21 [00:07<00:00,  2.71it/s]
INFO:dspy.evaluate.evaluate:2024-08-17T12:38:27.949350Z [info     ] Average Metric: 21 / 21 (100.0%) [dspy.evaluate.evaluate] filename=evaluate.py lineno=200


Looking at the traces, []
Score: 100.0 for set: [4]
New best sscore: 100.0 for seed 1
Scores so far: [95.24, 85.71, 95.24, 80.95, 100.0]
Best score: 100.0
Average of max per entry across top 1 scores: 1.0
Average of max per entry across top 2 scores: 1.0
Average of max per entry across top 3 scores: 1.0
Average of max per entry across top 5 scores: 1.0
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
5 candidate programs found.
